### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [2]:
import pandas as pd
import requests as re
import geopandas

In [3]:
#LEEMOS NUESTRO FICHERO
bicimad_df = pd.read_csv('../data/bicimad_stations.csv', sep = '\t')

In [4]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [5]:
bicimad_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            264 non-null    int64 
 1   id                    264 non-null    int64 
 2   name                  264 non-null    object
 3   light                 264 non-null    int64 
 4   number                264 non-null    object
 5   address               264 non-null    object
 6   activate              264 non-null    int64 
 7   no_available          264 non-null    int64 
 8   total_bases           264 non-null    int64 
 9   dock_bikes            264 non-null    int64 
 10  free_bases            264 non-null    int64 
 11  reservations_count    264 non-null    int64 
 12  geometry.type         264 non-null    object
 13  geometry.coordinates  264 non-null    object
dtypes: int64(9), object(5)
memory usage: 29.0+ KB


In [6]:
bicimad_df['longitude'] = bicimad_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[0], axis=1)

In [7]:
bicimad_df['latitude'] = bicimad_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[1], axis=1)

In [8]:
bicimad_df.head(1)

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.7018341,40.4172137


In [9]:
bicipark_df = pd.read_csv('../data/bicipark_stations.csv', sep=';')

In [10]:
bicipark_df['longitude'] = bicipark_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[0], axis=1)
bicipark_df['latitude'] = bicipark_df.apply(lambda row: row['geometry.coordinates'].strip('[]').split(', ')[1], axis=1)

In [14]:
bicipark_df[['longitude', 'latitude']] = bicipark_df.apply(lambda row: pd.Series(
    [row['geometry.coordinates'].strip('[]').split(', ')[0], 
row['geometry.coordinates'].strip('[]').split(', ')[1]]), axis=1)

In [12]:
bicipark_df.head(2)

,Unnamed: 0,stationId,stationName,address,city,zip_code,total_places,free_places,enabled,reserved_places,geometry.type,geometry.coordinates,longitude,latitude
0,0,74,Bicipark Fuente de la Mora,Estación de Fuente de la Mora,Madrid,28050,10,9,1,0,Point,"[-3.6630679, 40.4846838]",-3.6630679,40.4846838
1,1,22,Bicipark Orense,"Avenida General Perón, 27",Madrid,28020,15,9,1,1,Point,"[-3.693897, 40.452973]",-3.693897,40.452973


In [12]:
#CONECTAMOS CON LA WEB PARA OBTENER JSON
base_url = "https://datos.madrid.es/egob"
body = "/catalogo/209434-0-templos-otros.json"
response = re.get(base_url + body)
print(response)

<Response [200]>


In [13]:
#ANALIZAMOS JSON
content = response.content
json_data = response.json()
json_data.keys()

dict_keys(['@context', '@graph'])

In [14]:
json_data.keys()

dict_keys(['@context', '@graph'])

In [15]:
df1 = pd.DataFrame(json_data['@graph'])

In [16]:
def separator_longitud_json(column): #devuelve valor como float64
    for i in column:
        return column["longitude"]
def separator_latitud_json(column): #devuelve valor como float64
    for i in column:
        return column["latitude"]

In [17]:
df1['Longitud'] = df1.apply(lambda row: separator_longitud_json(row["location"]), axis=1)
df1['Latitud'] = df1.apply(lambda row: separator_latitud_json(row["location"]), axis=1)

In [18]:
df1.head(3)

,@id,@type,id,title,relation,address,location,organization,Longitud,Latitud
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11772877,Asamblea Cristiana Gilgal,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.38724931072868, 'longitude': -...",{'organization-desc': 'Templo evangélico. Bus...,-3.638521,40.387249
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11025958,Asambleas de Evangelización Mundial para Crist...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.372800292475574, 'longitude': ...",{'organization-desc': 'Templo evangélico Metr...,-3.745074,40.372800
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11026848,Asociación Evangélica Internacional,http://www.madrid.es/sites/v/index.jsp?vgnextc...,{'district': {'@id': 'https://datos.madrid.es/...,"{'latitude': 40.457895405861926, 'longitude': ...",{'organization-desc': 'Templo evangélico. Met...,-3.670675,40.457895


In [19]:
df_templos = df1[['id','title','Longitud','Latitud']]

In [20]:
bicimad_df.head(1)

,Unnamed: 0,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates,longitude,latitude
0,0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]",-3.7018341,40.4172137


In [21]:
bicimad_df = bicimad_df[['id','address','longitude','latitude']]